In [2]:
#import dependencies
import pandas as pd

In [4]:
#open dataset
schools_csv = "Resources/schools_complete.csv"
students_csv = "Resources/students_complete.csv"

schools_df = pd.read_csv(schools_csv, index_col="School ID")
students_df = pd.read_csv(students_csv, index_col="Student ID")
school_students_df = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])

In [5]:
#Check dataset consistency and stats
print(schools_df.describe())
print(schools_df.dtypes)
print(schools_df.count())
print(schools_df.head())

              size        budget
count    15.000000  1.500000e+01
mean   2611.333333  1.643295e+06
std    1420.915282  9.347763e+05
min     427.000000  2.480870e+05
25%    1698.000000  1.046265e+06
50%    2283.000000  1.319574e+06
75%    3474.000000  2.228999e+06
max    4976.000000  3.124928e+06
school_name    object
type           object
size            int64
budget          int64
dtype: object
school_name    15
type           15
size           15
budget         15
dtype: int64
                     school_name      type  size   budget
School ID                                                
0              Huang High School  District  2917  1910635
1           Figueroa High School  District  2949  1884411
2            Shelton High School   Charter  1761  1056600
3          Hernandez High School  District  4635  3022020
4            Griffin High School   Charter  1468   917500


In [6]:
#Check dataset consistency and stats
print(students_df.describe())
print(students_df.dtypes)
print(students_df.count())
print(students_df.head())

       reading_score    math_score
count    39170.00000  39170.000000
mean        81.87784     78.985371
std         10.23958     12.309968
min         63.00000     55.000000
25%         73.00000     69.000000
50%         82.00000     79.000000
75%         91.00000     89.000000
max         99.00000     99.000000
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
dtype: object
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
dtype: int64
                 student_name gender grade        school_name  reading_score  \
Student ID                                                                     
0                Paul Bradley      M   9th  Huang High School             66   
1                Victor Smith      M  12th  Huang High School             94   
2             Kevin Rodriguez      M  12th  Huang High S

In [7]:
#Set pass mark
maths_pass = 70
reading_pass = 70

Primary analysis
================

District Summary
------------------------

In [15]:
#Total schools
total_schools = schools_df.school_name.count()

#Total students
total_students = students_df.school_name.count()

#Total budget
total_budget = schools_df.budget.sum()

#Average maths score
average_maths = students_df.math_score.mean()

#Average reading score
average_reading = students_df.reading_score.mean()

#% passing maths
pc_pass_maths = students_df[students_df.math_score >= maths_pass].count()["math_score"]*100/total_students

#% passing reading
pc_pass_read = students_df[students_df.reading_score >= reading_pass].count()["reading_score"]*100/total_students

#% overall passing
pc_pass_both = students_df[(students_df.math_score >= maths_pass) & (students_df.reading_score >= reading_pass)].count()["math_score"]*100/total_students

#Create summary table
summary_df = pd.DataFrame(
              {"Total Schools": total_schools, "Total Students": total_students, "Total Budget": total_budget, 
               "Average Maths Score": average_maths, "Average Reading Score": average_reading,
              "% passing maths": pc_pass_maths, "% passing reading": pc_pass_read,
              "% passing both": pc_pass_both}, index=[1])

#Adjust format of budget cell
summary_df["Total Budget"] = summary_df["Total Budget"].map("${:,.2f}".format)
summary_df

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% passing maths,% passing reading,% passing both
1,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


School Summary
------------------------

In [77]:
#Group by School Name
grouped_students = school_students_df.groupby(["school_name", "type"])
grouped_students_df = pd.DataFrame(grouped_students)

#Total students
school_students = grouped_students["school_name"].size()

#Total school budget
school_budget = grouped_students["budget"].mean()

#Per student budget
school_student_budget = school_budget/school_students

#Average maths score
school_average_maths = grouped_students["math_score"].mean()

#Average reading score
school_average_reading = grouped_students["reading_score"].mean()

#% Passing maths
#school_passing_maths = grouped_students.filter(lambda x: x["math_score"]) >= maths_pass.count()
#school_pass_maths = grouped_students_df.loc[grouped_students_df["math_score"] >= maths_pass, :]

#% passing reading


#% overall passing

school_summary = pd.DataFrame(
                    {"Total Students": school_students, 
                     "Total School Budget": school_budget, "Per Student Budget": school_student_budget,
                     "Average maths score": school_average_maths, "Average reading score": school_average_reading})
school_summary = school_summary.reset_index()
school_summary = school_summary.rename(columns={"school_name": "School Name", "type": "Type"})
school_summary

,School Name,Type,Total Students,Total School Budget,Per Student Budget,Average maths score,Average reading score
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699


Top performing schools
----------------------------------

In [80]:
#Sort school summary
school_summary = school_summary.sort_values("Average maths score", ascending=False)

#Select top 5
school_summary.head(5)

,School Name,Type,Total Students,Total School Budget,Per Student Budget,Average maths score,Average reading score
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988
14,Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000
12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930
11,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724


Bottom performing schools
-------------------------------------

In [81]:
#Sort school summary


#Select bottom 5
school_summary.tail(5)

,School Name,Type,Total Students,Total School Budget,Per Student Budget,Average maths score,Average reading score
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722


Secondary analysis
==================

Maths scores by grade
---------------------

In [91]:
#Extract scores by grade
scores_9th = students_df.loc[students_df["grade"] == "9th", :].groupby("school_name").mean()
scores_10th = students_df.loc[students_df["grade"] == "10th", :].groupby("school_name").mean()
scores_11th = students_df.loc[students_df["grade"] == "11th", :].groupby("school_name").mean()
scores_12th = students_df.loc[students_df["grade"] == "12th", :].groupby("school_name").mean()

#Define maths dataframe
maths_df = pd.DataFrame( {
                    "9th grade": scores_9th["math_score"],
                    "10th grade": scores_10th["math_score"],
                    "11th grade": scores_11th["math_score"],
                    "12th grade": scores_12th["math_score"]
})
maths_df

,9th grade,10th grade,11th grade,12th grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


Reading scores by grade
-----------------------

In [93]:
#Define maths dataframe
reading_df = pd.DataFrame( {
                    "9th grade": scores_9th["reading_score"],
                    "10th grade": scores_10th["reading_score"],
                    "11th grade": scores_11th["reading_score"],
                    "12th grade": scores_12th["reading_score"]
})
reading_df

,9th grade,10th grade,11th grade,12th grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


Scores by school spending
-------------------------

In [ ]:
#Define bins


#Select data


#Bin data

Scores by school size
---------------------

In [ ]:
#Define bins


#Select data


#Bin data

Scores by school type
---------------------

In [ ]:
#Define bins


#Select data


#Bin data

Reporting
=========